In [ ]:
import cv2
import numpy as np

# Fungsi untuk mendeteksi warna dalam bounding box tetap di tengah layar
def detect_color_in_box(hsv_frame, color_ranges, x, y, w, h):
    # Memotong area di dalam bounding box tetap
    roi = hsv_frame[y:y+h, x:x+w]
    detected_color = None

    # Mengecek setiap rentang warna dalam color_ranges
    for color_name, (lower, upper) in color_ranges.items():
        # Membuat masker untuk rentang warna tertentu
        mask = cv2.inRange(roi, np.array(lower, np.uint8), np.array(upper, np.uint8))

        # Jika ada warna yang terdeteksi di dalam bounding box
        if cv2.countNonZero(mask) > 0:
            detected_color = color_name
            break

    return detected_color

# Rentang warna dalam HSV
color_ranges = {
    "Hitam": [(0, 0, 0), (180, 255, 50)],
    "Abu-abu": [(0, 0, 51), (180, 10, 200)],
    "Putih": [(0, 0, 201), (180, 20, 255)],
    "Merah": [(0, 50, 50), (10, 255, 255)],
    "Jingga": [(11, 50, 50), (25, 255, 255)],
    "Kuning": [(26, 50, 50), (35, 255, 255)],
    "Hijau": [(36, 50, 50), (85, 255, 255)],
    "Cyan": [(86, 50, 50), (95, 255, 255)],
    "Biru": [(96, 50, 50), (130, 255, 255)],
    "Ungu": [(131, 50, 50), (145, 255, 255)],
    "Magenta": [(146, 50, 50), (160, 255, 255)],
    "Coklat": [(10, 50, 20), (20, 200, 200)]
}

# Mengakses kamera
cap = cv2.VideoCapture(0)

# Ukuran bounding box tetap di tengah layar
box_width, box_height = 200, 200

while cap.isOpened():
    # Membaca frame dari kamera
    ret, frame = cap.read()
    if not ret:
        print("Tidak dapat mengambil frame dari kamera.")
        break

    # Mendapatkan dimensi frame dan menentukan posisi bounding box
    h, w, _ = frame.shape
    x_center = w // 2 - box_width // 2
    y_center = h // 2 - box_height // 2

    # Mengkonversi frame dari BGR ke HSV untuk deteksi warna yang lebih akurat
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Deteksi warna dalam bounding box tetap di tengah layar
    color_detected = detect_color_in_box(hsv_frame, color_ranges, x_center, y_center, box_width, box_height)

    # Menggambar bounding box di tengah layar
    cv2.rectangle(frame, (x_center, y_center), (x_center + box_width, y_center + box_height), (0, 0, 0), 2)
    
    # Menampilkan nama warna yang terdeteksi jika ada
    if color_detected:
        cv2.putText(frame, f"Warna: {color_detected}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Debugging: Menampilkan masker setiap warna untuk validasi
    for color_name, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv_frame, np.array(lower, np.uint8), np.array(upper, np.uint8))
        cv2.imshow(f"Mask {color_name}", mask)

    # Menampilkan frame kamera dengan hasil deteksi warna
    cv2.imshow("Deteksi Warna di Bounding Box Tetap", frame)

    # Tekan 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Debugging: Menampilkan masker setiap warna untuk validasi
    for color_name, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv_frame, np.array(lower, np.uint8), np.array(upper, np.uint8))
        cv2.imshow(f"Mask {color_name}", mask)

# Melepas kamera dan menutup jendela
cap.release()
cv2.destroyAllWindows()